In [7]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

# Define the domain class
class Domain:
    def __init__(self, x_min, x_max, t_min, t_max, num_x, num_t, viscosity):
        self.x_min = x_min
        self.x_max = x_max
        self.t_min = t_min
        self.t_max = t_max
        self.num_x = num_x
        self.num_t = num_t
        self.viscosity = viscosity

# Define the initial condition
def initial_condition(domain, x):
    return -np.sin(np.pi * x)

# Define the boundary conditions
def boundary_conditions(domain, x_boundary, x, t):
    return np.zeros_like(x)

# Define the physics-informed loss function
def physics_informed_loss(model, domain, x, t):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(x)
        tape.watch(t)

        u = model(x, t)
        u_x = tape.gradient(u, x)
        u_t = tape.gradient(u, t)

        print("u_x", u_x)
        print("u_t", u_t)

        # Define the Burgers equation residual
        residual = u_t + u * u_x - domain.viscosity * tape.gradient(u_x, x)
        # this is going to transform variables to a Tensor

        # Define the boundary and initial condition residuals
        initial_residual = u - initial_condition(domain, x)
        lower_boundary_residual = u - boundary_conditions(domain, domain.x_min, x, t)
        upper_boundary_residual = u - boundary_conditions(domain, domain.x_max, x, t)

    loss = tf.reduce_mean(tf.square(residual)) + \
           tf.reduce_mean(tf.square(initial_residual)) + \
           tf.reduce_mean(tf.square(lower_boundary_residual)) + \
           tf.reduce_mean(tf.square(upper_boundary_residual))

    return loss

# Define the neural network model
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,)),
        tf.keras.layers.Dense(32, activation='tanh'),
        tf.keras.layers.Dense(32, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    return model

# Train the model using PINNs
def train_model(model, domain, num_epochs, learning_rate, batch_size):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

    for epoch in range(num_epochs):
        x_batch = tf.random.uniform([batch_size, 1], minval=domain.x_min, maxval=domain.x_max)
        t_batch = tf.random.uniform([batch_size, 1], minval=domain.t_min, maxval=domain.t_max)
        
        with tf.GradientTape() as tape:
            loss = physics_informed_loss(model, domain, x_batch, t_batch)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, Loss: {loss.numpy()}")

    return model

# Visualize the results
def visualize_results(model, domain):
    x_grid, t_grid = np.meshgrid(np.linspace(domain.x_min, domain.x_max, domain.num_x),
                                 np.linspace(domain.t_min, domain.t_max, domain.num_t))
    X = np.hstack((x_grid.flatten()[:, np.newaxis], t_grid.flatten()[:, np.newaxis]))
    u_pred_grid = model.predict(X)
    u_pred_grid = griddata(X, u_pred_grid.flatten(), (x_grid, t_grid), method='cubic')

    plt.figure(figsize=(8, 6))
    plt.pcolor(x_grid, t_grid, u_pred_grid, cmap='jet')
    plt.colorbar()
    plt.xlabel('x')
    plt.ylabel('t')
    plt.title('Predicted Velocity')
    plt.show()

if __name__ == "__main__":
    # Define domain parameters and create the domain
    x_min = -1.0
    x_max = 1.0
    t_min = 0.0
    t_max = 0.99
    num_x = 100
    num_t = 100
    viscosity = 0.01
    domain = Domain(x_min, x_max, t_min, t_max, num_x, num_t, viscosity)

    # Create the neural network 
    model = create_model()

    # Train the model using PINNs
    model = train_model(model, domain, num_epochs=2000, learning_rate=0.01, batch_size=32)

    # Visualize the results
    visualize_results(model, domain)


u_x tf.Tensor(
[[0.35362712]
 [0.34899762]
 [0.33561212]
 [0.3459878 ]
 [0.35398376]
 [0.3523803 ]
 [0.3553429 ]
 [0.32603762]
 [0.32496783]
 [0.34658563]
 [0.33789608]
 [0.3249273 ]
 [0.34429562]
 [0.34345073]
 [0.35193306]
 [0.3564657 ]
 [0.34596062]
 [0.35375854]
 [0.32730955]
 [0.35665745]
 [0.35456026]
 [0.32884908]
 [0.3572298 ]
 [0.32768416]
 [0.31209248]
 [0.34915388]
 [0.32192713]
 [0.35716164]
 [0.35686296]
 [0.3386773 ]
 [0.35589674]
 [0.33904314]], shape=(32, 1), dtype=float32)
u_t None


ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.